# 1. 모델 구현

* https://www.tensorflow.org/guide/keras/sequential_model

```
#Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"), # Dence : 일반 레이어
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
#Call model on a test input
x = tf.ones((3, 3))
y = model(x)
```

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# 모델을 만들어주는 함수 정의
def get_sequential_model(imput_shape):
    model = keras.Sequential(
        [
            layers.Input(input_shape), # 입력층 만듦 -> 은닉층으로 데이터를 보낼거임
            
            # 1st
            # 64개의 데이터 받음(필터 64개 사용). 3x3 필터크기. 패딩을 자동으로 삽입해서 크기 유지(<-> 'valid')
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            # 가중치 2번 업데이트. 좀 더 세밀해짐.
            layers.MaxPool2D(), # 가장 큰 값을 뽑아서 크기를 줄여줌
            layers.BatchNormalization(), # 픽셀값을 정규화.
            layers.Dropout(0.5), # 과대적합 방지를 위해
            
            # 2nd
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'), # 128개 데이터 추출
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(), 
            layers.BatchNormalization(), 
            layers.Dropout(0.3),
            
            # FC
            layers.GlobalMaxPool2D(), # 다합쳐서 맥스풀링
            layers.Dense(128, activation="relu"), # Dence : 일반 레이어
            layers.Dense(1, activation='sigmoid') # 참/거짓 둘 중 하나로 값 추출되도록
        ]
    )
    return model

In [6]:
input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 conv2d_5 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 128, 128, 64)     256       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 128, 128, 64)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 128, 128, 128)    

In [8]:
class SimpleCNN(keras.Model): # keras.Model을 상속받아옴
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv_block_1 = keras.Sequential(
            [
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.5)
            ], name='conv_block_1'
        )

        self.conv_block_2 = keras.Sequential(
            [
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.3)
            ], name='conv_block_2'
        )
        
        self.classifier = keras.Sequential(
            [
                layers.GlobalMaxPool2D(),
                layers.Dense(128, activation="relu"),
                layers.Dense(1, activation='sigmoid')
            ], name='classifier'
        )
    
    def call(self, input_tensor, training=False):
        x = self.conv_block_1(input_tensor)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

In [11]:
input_shape = (None, 256, 256, 3) # batch_siz는 None
model = SimpleCNN()
model.build(input_shape)

model.summary()

Model: "simple_cnn_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block_1 (Sequential)   (None, 128, 128, 64)      38976     
                                                                 
 conv_block_2 (Sequential)   (None, 64, 64, 128)       221952    
                                                                 
 classifier (Sequential)     (None, 1)                 16641     
                                                                 
Total params: 277,569
Trainable params: 277,185
Non-trainable params: 384
_________________________________________________________________


In [12]:
# model에 setting하는 함수
model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics='accuracy' # 검증방법
)